## Import libraries

In [15]:
# Standard libraries
import os
import json
import sqlite3
from pathlib import Path

# Data handling
import pandas as pd
import numpy as np

from sams.config import datasets
from sams.utils import load_data

from IPython.display import display_html

import duckdb as ddb
import importlib
import sams.utils
importlib.reload(sams.utils)  
# from sams.utils import decrypt_roll

import re

In [ ]:
# Table: HSS vs DEG Enrollment by Academic Year (Aadhaar and Student Key Matching)
# Columns: year, hss_unique_aadhar, hss_1by1_match_addahr_with_key, deg_unique_aadhar, deg_1by1_match_addahr_with_key

query = """
WITH hss AS (
    SELECT academic_year AS year, aadhar_no, student_key
    FROM enr_hss
    WHERE aadhar_no IS NOT NULL AND aadhar_no != '' AND student_key IS NOT NULL AND student_key != ''
),
deg AS (
    SELECT academic_year AS year, aadhar_no, student_key
    FROM enr_deg
    WHERE aadhar_no IS NOT NULL AND aadhar_no != '' AND student_key IS NOT NULL AND student_key != ''
),
hss_aadhar AS (
    SELECT year, COUNT(DISTINCT aadhar_no) AS hss_unique_aadhar
    FROM hss
    GROUP BY year
),
hss_match AS (
    SELECT year, COUNT(*) AS hss_1by1_match_addahr_with_key
    FROM (
        SELECT year, aadhar_no, student_key
        FROM hss
        GROUP BY year, aadhar_no, student_key
        HAVING COUNT(*) = 1
    )
    GROUP BY year
),
deg_aadhar AS (
    SELECT year, COUNT(DISTINCT aadhar_no) AS deg_unique_aadhar
    FROM deg
    GROUP BY year
),
deg_match AS (
    SELECT year, COUNT(*) AS deg_1by1_match_addahr_with_key
    FROM (
        SELECT year, aadhar_no, student_key
        FROM deg
        GROUP BY year, aadhar_no, student_key
        HAVING COUNT(*) = 1
    )
    GROUP BY year
)
SELECT 
    COALESCE(h.year, d.year) AS year,
    h.hss_unique_aadhar,
    hm.hss_1by1_match_addahr_with_key,
    d.deg_unique_aadhar,
    dm.deg_1by1_match_addahr_with_key
FROM hss_aadhar h
LEFT JOIN hss_match hm ON h.year = hm.year
FULL OUTER JOIN deg_aadhar d ON h.year = d.year OR d.year IS NOT NULL
LEFT JOIN deg_match dm ON d.year = dm.year
ORDER BY year;
"""

df = con.sql(query).df()
display_html(df, raw=True)
df

## Step 01 — Register dataset views 

In [3]:

con = ddb.connect("sams.duckdb")
con.sql("PRAGMA memory_limit='12GB'; PRAGMA threads=8;")

def view(name, key):
    path = str(datasets[key]["path"])
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    con.sql(f"CREATE OR REPLACE VIEW {name} AS SELECT * FROM read_parquet('{path}');")

for name, key in [
    ("enr_deg", "deg_enrollments"),
    ("enr_hss", "hss_enrollments"),
    ("app_deg", "deg_applications"),
    ("app_hss", "hss_applications"),
]:
    view(name, key)

# quick sanity peek (optional)
con.sql("SELECT * FROM enr_deg LIMIT 5").show()

┌───────┬───────────┬──────────────────────────────────────────────┬─────────┬───────────────┬────────────┬───────────────┬───────────────────────────────────────────────────────┬─────────┬────────────┬──────────────────┬──────────┬─────────────────┬───────────────────────────────────────────────┬───────────────────────┬─────────┬─────────┬─────────┬─────────┬───────────┬─────────┬───────────────┬────────────────────────────────────────────────┬──────────────────┬─────────────────┬──────────────────────────┬─────────────┬───────────────┬────────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [4]:
con.sql("SELECT * FROM app_deg LIMIT 5").df()

,barcode,aadhar_no,academic_year,year,phase,reported_institute,sams_code,institute_district,institute_block,type_of_institute,stream,subject,option_no,admission_status,num_applications
0,18D000005,None,2018,2018,1,"Basic Science & Humanities (Degree) College, B...",19056302,Khurda,Bhubaneswar (MC),Govt. (Other Depts.),Self Financing,Computer Science,1,NOT SELECTED,10
1,18D000005,None,2018,2018,1,"Prananath (Autonomous) College, Khurda",19152904,Khurda,Khordha (MPL),Non-Govt Aided (488 Categories),Self Financing,Computer Science,2,TC TAKEN,10
2,18D000005,None,2018,2018,1,Buxi Jagabandhu Bidyadhar (Autonomous) College...,19051908,Khurda,Bhubaneswar (MC),Govt. (SME),Self Financing,Computer Science,3,NOT SELECTED,10
3,18D000005,None,2018,2018,1,Buxi Jagabandhu Bidyadhar (Autonomous) College...,19051908,Khurda,Bhubaneswar (MC),Govt. (SME),Self Financing,Biotechnology,4,NOT SELECTED,10
4,18D000005,None,2018,2018,1,"Basic Science & Humanities (Degree) College, B...",19056302,Khurda,Bhubaneswar (MC),Govt. (Other Depts.),Self Financing,Biotechnology,5,NOT SELECTED,10


In [13]:
# can we print the highest qualification count for enr_deg?
# Highest qualification count for enr_deg
con.sql("""
SELECT highest_qualification, COUNT(*) AS qualification_count
FROM enr_deg
GROUP BY highest_qualification
ORDER BY qualification_count DESC
""").df()

,highest_qualification,qualification_count
0,+2 ARTS,1153670
1,+2 SCIENCE,662733
2,+2 COMMERCE,172320
3,+2 UPASASTRI,31394
4,+2 VOCATIONAL,26648
5,DIPLOMA,7651
6,None,75


In [3]:
iti_applications = pd.read_parquet("iti_applications.pq")

In [14]:
iti_enrollments.head()

NameError: name 'iti_enrollments' is not defined

In [10]:
iti_applications.columns

Index(['barcode', 'aadhar_no', 'gender', 'dob', 'annual_income', 'state',
       'district', 'social_category', 'domicile', 'date_of_application',
       'academic_year', 'year', 'course_name', 'course_period',
       'reported_institute', 'reported_branch_or_trade', 'institute_district',
       'type_of_institute', 'highest_qualification', 'gc', 'ph', 'es',
       'sports', 'national_cadet_corps', 'orphan', 'ews', 'admission_status',
       'enrollment_status', 'Phase', 'Option_No', 'institute_name', 'trade',
       'status', 'num_applications'],
      dtype='object')

## Step 02— Load Diploma students from SQLite and expand mark_data


In [43]:
# Correct way to detach sqlite db if already attached
try:
    con.sql("DETACH DATABASE sams;")
except:
    pass


In [44]:
# === Build enr_diploma and enr_iti from SQLite (short, memory-efficient) ===

import duckdb as ddb, os

# connect once
con = con if 'con' in globals() else ddb.connect("sams.duckdb")
con.sql("PRAGMA memory_limit='12GB'; PRAGMA threads=8;")

def esc(s): return s.replace("'", "''")
db_path = str(datasets["sams"]["path"])

# --- Safe re-attach SQLite SAMS DB ---
try:
    con.sql("DETACH DATABASE sams;")
except:
    pass

con.sql("INSTALL sqlite_scanner; LOAD sqlite_scanner;")
con.sql(f"ATTACH DATABASE '{esc(db_path)}' AS sams (TYPE SQLITE);")

# --- Function to build enrollment datasets ---
def make_enr(mod, out):
    con.sql(f"""
    CREATE OR REPLACE TABLE {out} AS
    WITH src AS (
        SELECT student_name, aadhar_no, academic_year, dob, module, mark_data
        FROM sams.students
        WHERE module = '{mod}'
    ),
    exploded AS (
        SELECT student_name, aadhar_no, academic_year, dob, module,
               je.value AS md
        FROM src,
             LATERAL json_each(CASE WHEN json_valid(mark_data) THEN mark_data ELSE '[]' END) AS je
    )
    SELECT
        academic_year,
        dob,
        json_extract_scalar(md, '$.HighestQualificationBoardExamName') AS exam_board,
        module,
        CAST(NULLIF(json_extract_scalar(md, '$.YearofPassing'), '') AS INTEGER) AS passing_year,
        json_extract_scalar(md, '$.RollNo') AS roll_no,
        NULL AS roll_no_decrypted,
        NULL AS student_key,
        student_name
    FROM exploded;
    """)

# --- Build Diploma & ITI tables ---
make_enr("Diploma", "enr_diploma")
make_enr("ITI", "enr_iti")

# --- Preview a few rows ---
print("Diploma Applications:")
con.sql("SELECT * FROM enr_diploma LIMIT 5").show()

print("\nITI Applications:")
con.sql("SELECT * FROM enr_iti LIMIT 5").show()


BinderException: Binder Error: Failed to attach database: database with name "sams" already exists

In [18]:
db_path = datasets["sams"]["path"]   
conn = sqlite3.connect(db_path)

query = """
SELECT
    student_name,
    aadhar_no,
    academic_year,
    dob,
    mark_data,
    module
FROM students
WHERE module = 'Diploma';
"""

students_df = pd.read_sql_query(query, conn)
conn.close()

# Function to parse JSON strings safely
def parse_option_data(val):
    """
    Safely parses a JSON string into a Python object.

    Parameters:
    ----------
    val : str, list, or any
        The value to be parsed. If it's a JSON string, it will be decoded.
        If it's already a list, it is returned unchanged.
        All other types return an empty list.

    Returns:
    -------
    list
        Parsed list from JSON string, or the original list, or an empty list if parsing fails.
    """
    if isinstance(val, str):
        try:
            return json.loads(val)
        except json.JSONDecodeError:
            return []
    elif isinstance(val, list):
        return val
    else:
        return []

students_df['mark_data'] = students_df['mark_data'].apply(parse_option_data)
df_exploded = students_df.explode('mark_data').reset_index(drop=True)
option_details = pd.json_normalize(df_exploded['mark_data'])
diploma_applications = pd.concat([df_exploded.drop(columns=['mark_data']), option_details], axis=1)

In [20]:
diploma_applications.head()

,student_name,aadhar_no,academic_year,dob,module,ExamName,YearofPassing,RollNo,ExaminationType,TotalMarks,SecuredMarks,Percentage,CompartmentalStatus,CompartmentalFailMark,SubjectWiseMarks,HighestQualificationBoardExamName,HighestQualificationExamBoard
0,YADUNATH PRADHAN,yuHyzmPe2cGIN3TsA11RV51PLvfLYcoUGMmJFKP7xAI=,2018,30-Jan-2001,Diploma,10th,2017,+89E1khgPGeQc9e4DXSKcw==,Annual,600.00,370.00,61.67000000000000,No,NA,"MIL Total :100.00 MIL Secured :89.00,Eng Total...",NA,Indian Certificate of Secondary Education (ICS...
1,SUSANTA PATRA,AgPzPgZCw33tK6EvZqs8/KAXaALWLdyhMJzxGSUc/d0=,2018,12-Feb-2003,Diploma,10th,2018,4pVlT4koCAmJi1nS5e5/sA==,Annual,0.00,0.00,0.00000000000000,No,NA,"MIL Total :0.00 MIL Secured :0.00,Eng Total :0...",NA,"Board of Secondary Education, Odisha, Cuttack-..."
2,Ayush Acharya,2pJKsU1iZq9ipSiLbch3upa5qaTgxpU73tihGR5Bzy0=,2018,08-Feb-2002,Diploma,10th,2017,p0CFuzIOwIlZL5AXWsqFFg==,Annual,10.00,9.40,94.00000000000000,No,NA,"MIL Total :0.00 MIL Secured :0.00,Eng Total :1...",NA,"Central Board of Secondary Education, 02-Commu..."
3,SOTRAJIT DAS,qdWd2luTGhCcAHGzes9axW+7rnfpE3v+58IL32zTiwg=,2018,22-Oct-2001,Diploma,10th,2018,EGgDsHyh+g3rEBxzK0J5hQ==,Annual,600.00,431.00,71.83000000000000,No,NA,"MIL Total :100.00 MIL Secured :87.00,Eng Total...",NA,Indian Certificate of Secondary Education (ICS...
4,NILESH KUMAR,nZiMIc/6eGufdWeUY+NM3yofkeFn3CSxosQeGLyXSyw=,2018,19-Oct-2000,Diploma,10th,2016,hMQNQvyxRW8uVUz3V/WVbg==,Annual,10.00,8.60,86.00000000000000,No,NA,"MIL Total :0.00 MIL Secured :0.00,Eng Total :1...",NA,"Central Board of Secondary Education, 02-Commu..."


In [21]:
diploma_applications.columns

Index(['student_name', 'aadhar_no', 'academic_year', 'dob', 'module',
       'ExamName', 'YearofPassing', 'RollNo', 'ExaminationType', 'TotalMarks',
       'SecuredMarks', 'Percentage', 'CompartmentalStatus',
       'CompartmentalFailMark', 'SubjectWiseMarks',
       'HighestQualificationBoardExamName', 'HighestQualificationExamBoard'],
      dtype='object')

In [22]:
iti_applications.columns

Index(['barcode', 'aadhar_no', 'gender', 'dob', 'annual_income', 'state',
       'district', 'social_category', 'domicile', 'date_of_application',
       'academic_year', 'year', 'course_name', 'course_period',
       'reported_institute', 'reported_branch_or_trade', 'institute_district',
       'type_of_institute', 'highest_qualification', 'gc', 'ph', 'es',
       'sports', 'national_cadet_corps', 'orphan', 'ews', 'admission_status',
       'enrollment_status', 'Phase', 'Option_No', 'institute_name', 'trade',
       'status', 'num_applications'],
      dtype='object')

In [29]:
# keep only desired columns for ITI + Diploma applications (short & memory-safe)

# desired master columns
MASTER = ['academic_year','dob','exam_board','module','passing_year',
          'roll_no','roll_no_decrypted','student_key','student_name']

def project_app(src, dst, select_map: dict, const: dict):
    # build SELECT list from source→target map + constants
    sel = []
    for tgt in MASTER:
        if tgt in select_map:        # from a source column/expression
            sel.append(f"{select_map[tgt]} AS {tgt}")
        elif tgt in const:           # constant value
            sel.append(f"{const[tgt]} AS {tgt}")
        else:                        # not available yet → NULL
            sel.append(f"NULL AS {tgt}")
    con.sql(f"CREATE OR REPLACE TABLE {dst} AS SELECT {', '.join(sel)} FROM {src};")

# ---- Diploma Applications ----
# columns seen: student_name, aadhar_no, academic_year, dob, module, YearofPassing, RollNo,
#               HighestQualificationBoardExamName, HighestQualificationExamBoard, ...
project_app(
    src="diploma_applications",
    dst="diploma_app_clean",
    select_map={
        'exam_board':    "HighestQualificationExamBoard",
        'passing_year':  '"YearofPassing"',
        'roll_no':       '"RollNo"',
    },
    const={}  # no constants needed here
)

# ---- ITI Applications ----
# columns seen: academic_year, dob, year, (no student_name, no exam_board, no roll_no)
project_app(
    src="enr_iti",                    # if your ITI applications view has a different name, change here
    dst="iti_app_clean",
    select_map={
        'academic_year': 'academic_year',
        'dob':           'dob',
        'passing_year':  'year',
    },
    const={
        'module':        "'ITI'",     # set module explicitly
    }
)

# quick heads to verify
print("Diploma Applications (clean)")
con.sql("SELECT * FROM diploma_app_clean LIMIT 5").show()

print("\nITI Applications (clean)")
con.sql("SELECT * FROM iti_app_clean LIMIT 5").show()


Diploma Applications (clean)
┌───────────────┬───────┬───────────────────────────────────────────────────────────────────────────────────────────────────────┬────────┬──────────────┬──────────────────────────┬───────────────────┬─────────────┬──────────────┐
│ academic_year │  dob  │                                              exam_board                                               │ module │ passing_year │         roll_no          │ roll_no_decrypted │ student_key │ student_name │
│     int32     │ int32 │                                                varchar                                                │ int32  │   varchar    │         varchar          │       int32       │    int32    │    int32     │
├───────────────┼───────┼───────────────────────────────────────────────────────────────────────────────────────────────────────┼────────┼──────────────┼──────────────────────────┼───────────────────┼─────────────┼──────────────┤
│          NULL │  NULL │ Indian Certificate of Sec

,barcode,aadhar_no,gender,dob,annual_income,state,district,social_category,domicile,date_of_application,...,orphan,ews,admission_status,enrollment_status,Phase,Option_No,institute_name,trade,status,num_applications
0,17T0190772,YTdJgeEX26Ycl8mWb7DRsqB0vInZx4lhMXfrErnJWww=,Male,18-Apr-2000,"0-1,00,000",Odisha,Koraput,General,NA,02-Jun-2017,...,No,NA,No,No,1,1.0,Gopabandhu ITI Ambaguda Koraput,Electrician (NSQF),Opted,4
1,17T0190772,YTdJgeEX26Ycl8mWb7DRsqB0vInZx4lhMXfrErnJWww=,Male,18-Apr-2000,"0-1,00,000",Odisha,Koraput,General,NA,02-Jun-2017,...,No,NA,No,No,1,2.0,Gopabandhu ITI Ambaguda Koraput,Fitter (NSQF),Opted,4
2,17T0190772,YTdJgeEX26Ycl8mWb7DRsqB0vInZx4lhMXfrErnJWww=,Male,18-Apr-2000,"0-1,00,000",Odisha,Koraput,General,NA,02-Jun-2017,...,No,NA,No,No,1,3.0,Gopabandhu ITI Ambaguda Koraput,Machinist (NSQF),Opted,4
3,17T0190772,YTdJgeEX26Ycl8mWb7DRsqB0vInZx4lhMXfrErnJWww=,Male,18-Apr-2000,"0-1,00,000",Odisha,Koraput,General,NA,02-Jun-2017,...,No,NA,No,No,1,4.0,Gopabandhu ITI Ambaguda Koraput,Refrigeration &amp; Air Conditioning Technicia...,Opted,4
4,17T0434532,wlVLDFUWK3hDq+vrDsBrm1vx/vIUcsRP7yOmovfNifc=,Male,25-Sep-2000,"0-1,00,000",Odisha,Ganjam,General,NA,07-Aug-2017,...,No,NA,No,No,None,NaN,None,None,None,0


In [ ]:
# code for decryption
from base64 import b64decode
from Crypto.Cipher import AES

def decrypt_roll(enc_text: str,
                 key: bytes = b"y6idXfCVRG5t2dkeBnmHy9jLu6TEn5Du",
                 enforce_min_length: bool = False,
                 min_length: int = None) -> str:
    try:
        if not enc_text or not isinstance(enc_text, str):
            return "NA"

        raw = b64decode(enc_text)
        cipher = AES.new(key, AES.MODE_ECB)
        decrypted = cipher.decrypt(raw)

        pad_len = decrypted[-1]
        if pad_len < 1 or pad_len > 16:
            return "NA"
        decrypted = decrypted[:-pad_len]

        roll_no = decrypted.decode("utf-8").strip()
        return roll_no
    except Exception:
        return "NA"    

In [ ]:
def process_roll_numbers_len_format(df: pd.DataFrame, roll_col: str = 'roll_no') -> pd.DataFrame:
    """
    Decrypt roll numbers and validate only by length rule:
    - BSE Odisha: length must be 9
    - CHSE Odisha: length must be 8
    - Other boards: keep decrypted roll as-is
    """

    # Decrypt roll numbers
    df['roll_no_decrypted'] = df[roll_col].map(decrypt_roll)

    # Identify Odisha boards 
    board_col = df['exam_board'].fillna("NA").str.upper()
    # Put the condition to pass these input values of board name        
    mask_bse = (board_col.str.contains(r'\bBOARD OF SECONDARY EDUCATION,\s*ODISHA\b', regex=True)  
                | (board_col.str.contains(r'\bBSE\b(?! MADHYAMA).*ODISHA\b', regex=True) & ~board_col.str.contains(r'\bICSE\b|\bCBSE\b', regex=True)))
    
    mask_chse = (board_col.str.contains(r'\bCOUNCIL OF HIGHER SECONDARY EDUCATION,\s*ODISHA\b', regex=True) 
                 | board_col.str.contains(r'\bCHSE\b.*ODISHA\b', regex=True))

    # Apply validation
    if mask_bse.any():
        rolls_bse = df.loc[mask_bse & df['roll_no_decrypted'].notna(), 'roll_no_decrypted'].astype(str)
        valid_bse = rolls_bse.str.len() == 9
        df.loc[mask_bse & ~valid_bse, 'roll_no_decrypted'] = 'NA'

    if mask_chse.any():
        rolls_chse = df.loc[mask_chse & df['roll_no_decrypted'].notna(), 'roll_no_decrypted'].astype(str)
        valid_chse = rolls_chse.str.len() == 8
        df.loc[mask_chse & ~valid_chse, 'roll_no_decrypted'] = 'NA'

    return df